In [1]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=90907adb08ce6ffbc7dfe10924e7b2325760a81d86c11365eed539ab4606cf84
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/text_classification/dot/word2vec-transformerxl

/content/drive/MyDrive/text_classification/dot/word2vec-transformerxl


In [5]:
import torch
import pandas as pd
from transformers import TransfoXLModel, TransfoXLTokenizer
from torch.nn.utils.rnn import pad_sequence
import jieba

In [6]:
## transformerxl ##
# load data
data = pd.read_excel('./data.xlsx')
sentences = data['review'].tolist()

def txt_cut(s):
    res = [w for w in jieba.lcut(s) if w.strip()]
    return " ".join(res)

# tokenize
tokenized_text = []
for i in sentences:
    new = txt_cut(i).split(' ')
    tokenized_text.extend(new)

# load the Transformer-XL model and tokenizer
model = TransfoXLModel.from_pretrained('transfo-xl-wt103')
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')

# tokenize the reviews and convert them to input tensors
tokens = [tokenizer.encode(word, add_special_tokens=True) for word in tokenized_text]

# pad the sequences to have the same length
padded_tokens = pad_sequence([torch.tensor(token) for token in tokens], batch_first=True)

# get the word vectors
with torch.no_grad():
    outputs = model(padded_tokens)

word_vectors = outputs[0][:, -1, :]

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.469 seconds.
DEBUG:jieba:Loading model cost 1.469 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [7]:
import pandas as pd
from gensim.models import Word2Vec
import jieba
import numpy as np

In [8]:
## word2vec ##

# 初始化Word2Vec模型
model_w = Word2Vec(vector_size=100, window=5, min_count=1)

# 构建词汇表
model_w.build_vocab([tokenized_text])

# 训练模型
model_w.train([tokenized_text], total_examples=model_w.corpus_count, epochs=model_w.epochs)

# 创建一个空的DataFrame来保存词汇和对应的词向量
word_vector_df = pd.DataFrame()

for wd in model_w.wv.index_to_key:
    # 将词向量转化为Series，然后添加到DataFrame中
    word_vector_series = pd.Series([wd] + list(model_w.wv[wd]))
    word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)

<ipython-input-8-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-8-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-8-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython-input-8-f084292095fa>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  word_vector_df = word_vector_df.append(word_vector_series, ignore_index=True)
<ipython

In [9]:
new_word_vectors = []

# 获取词汇表
vocab = tokenizer.get_vocab()

# 创建字典，将词汇表中的单词与模型输出的词向量相对应
word_to_vector = {}

for i, word in enumerate(tokenized_text):
    word_to_vector[word] = word_vectors[i]

# 检查每个单词及其词向量
for word in tokenized_text:
    word_vector_t = word_to_vector[word]
    word_vector_t = word_vector_t.reshape(-1, 1)

    ## word2vec
    word_vector_w = model_w.wv[word]
    word_vector_w = word_vector_w.reshape(-1, 1)

    ## 新词向量
    new_word_vector = np.multiply(word_vector_t, word_vector_w.T)
    new_word_vectors.append((word, new_word_vector))

In [ ]:
# 指定保存文件路径
save_path = 'new_word_vectors.txt'

# 保存单词和新词向量到文件
with open(save_path, 'w') as f:
    for word, word_vector in new_word_vectors:
        word_vector_str = ','.join(map(str, word_vector))
        line = f"{word}: {word_vector_str}\n"
        f.write(line)